In [35]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_data_generators(root, img_size=(28, 28), batch_size=32, split=[0.9, 0.05, 0.05]):
    # Initialize ImageDataGenerators
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=split[1] + split[2]  # combined validation + test split
    )
    
    # Create generators
    train_generator = datagen.flow_from_directory(
        directory=root,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',  # Grayscale images
        subset='training',
        class_mode='sparse'
    )
    
    val_generator = datagen.flow_from_directory(
        directory=root,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',  # Grayscale images
        subset='validation',
        class_mode='sparse'
    )
    
    test_generator = datagen.flow_from_directory(
        directory=root,
        target_size=img_size,
        batch_size=1,
        color_mode='grayscale',  # Grayscale images
        subset='validation',
        class_mode='sparse'
    )
    
    return train_generator, val_generator, test_generator

# Example usage
root = r"C:\Users\bedir\OneDrive\Masaüstü\YMIR\PROJECTS\PYTHON\AI\SUDOKU_SOLVER\SUDOKU_SOLVER\Model\data\assets"
img_size = (28, 28)
batch_size = 32

train_gen, val_gen, test_gen = get_data_generators(root=root, img_size=img_size, batch_size=batch_size)

# Print number of batches
print("Number of training batches:", len(train_gen))
print("Number of validation batches:", len(val_gen))
print("Number of test batches:", len(test_gen))

# Print class names
print("Class names:", train_gen.class_indices)


Found 5672 images belonging to 10 classes.
Found 627 images belonging to 10 classes.
Found 627 images belonging to 10 classes.
Number of training batches: 178
Number of validation batches: 20
Number of test batches: 627
Class names: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [37]:
def create_sudoku_cnn_model():
    model = Sequential()
    
    # İlk Konvolüsyon Katmanı
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # İkinci Konvolüsyon Katmanı
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Üçüncü Konvolüsyon Katmanı
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Düzleştirme ve Yoğun Katmanlar
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # 10 sınıf için çıkış katmanı
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [38]:
model = create_sudoku_cnn_model()

history = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen
)

test_loss, test_acc = model.evaluate(test_gen)
print(f'Test Accuracy: {test_acc}')


Epoch 1/50
178/178 [==============================] - 10s 49ms/step - loss: 1.5393 - accuracy: 0.4441 - val_loss: 0.9484 - val_accuracy: 0.6986
Epoch 2/50
178/178 [==============================] - 4s 22ms/step - loss: 0.5888 - accuracy: 0.7872 - val_loss: 0.4539 - val_accuracy: 0.8660
Epoch 3/50
178/178 [==============================] - 4s 23ms/step - loss: 0.3256 - accuracy: 0.8898 - val_loss: 0.3546 - val_accuracy: 0.8708
Epoch 4/50
178/178 [==============================] - 3s 19ms/step - loss: 0.2142 - accuracy: 0.9291 - val_loss: 0.3043 - val_accuracy: 0.8947
Epoch 5/50
178/178 [==============================] - 3s 18ms/step - loss: 0.1602 - accuracy: 0.9490 - val_loss: 0.1947 - val_accuracy: 0.9394
Epoch 6/50
178/178 [==============================] - 3s 17ms/step - loss: 0.1333 - accuracy: 0.9559 - val_loss: 0.1964 - val_accuracy: 0.9410
Epoch 7/50
178/178 [==============================] - 3s 18ms/step - loss: 0.1137 - accuracy: 0.9642 - val_loss: 0.1310 - val_accuracy: 0.963

In [33]:
def save_model(model, path):
    model.save(path)
    print(f"Model saved to {path}")

In [34]:
save_path = r'C:\Users\bedir\Models\sudoku_solver_model_better.h5'
save_model(model, save_path)


Model saved to C:\Users\bedir\Models\sudoku_solver_model.h5
